In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
#train_dataset = datasets.MNIST(root='/root/data/', train=True, transform=transforms.ToTensor(), download=True)
train_dataset = datasets.MNIST(root='/root/data/', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = datasets.MNIST(root='/root/data/', train=False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [2]:
import torch
import torch.nn as nn



class VQVAE(nn.Module):

    def __init__(self, back_kernels = [3, 3], channels = [1, 64, 256], latent_dim = 2, n_embedding = 10) -> None:
        super().__init__()
        
        
        # encoder
        pre_channel = 1
        modules = []
        img_length = 28
        for i in range(len(back_kernels)):
            modules.append(
                nn.Sequential(
                    nn.Conv2d(pre_channel,
                              channels[i+1],
                              kernel_size=3,
                              stride=2,
                              padding=1),
                    nn.BatchNorm2d(channels[i+1]),
                    nn.ReLU()
                )
            )
            pre_channel = channels[i+1]
            img_length = (img_length-1)//2+1
        modules.append(
            nn.Sequential(
                nn.Conv2d(pre_channel,
                          latent_dim,
                          kernel_size=3,
                          stride=1,
                          padding=1),
            )
        )
        self.encoder = nn.Sequential(*modules)
        self.vq_embedding = nn.Embedding(n_embedding, latent_dim)
        self.vq_embedding.weight.data.uniform_(-1.0 / n_embedding, 1.0 / n_embedding)
        
        # decoder
        modules = []
        modules.append(
            nn.Sequential(
                nn.Conv2d(latent_dim,
                          pre_channel,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.BatchNorm2d(pre_channel),
                nn.ReLU()
            )
        )
        for i in range(len(back_kernels)):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(channels[len(back_kernels)-i],
                                       channels[len(back_kernels)-i-1],
                                       kernel_size=back_kernels[i],
                                       stride=2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(channels[len(back_kernels)-i-1]),
                    nn.ReLU()
                )
            )
        self.decoder = nn.Sequential(*modules)
        

    def forward(self, x):
        ze = self.encoder(x)
        # ze: [Num, Channel, Height, Width]
        # embedding: [emb_num, Channel]
        embedding = self.vq_embedding.weight.data
        Num, Channel, Height, Width = ze.shape
        emb_num, _ = embedding.shape
        embedding_broad = embedding.reshape(1, emb_num, Channel, 1, 1)
        ze_broad = ze.reshape(Num, 1, Channel, Height, Width)
        distance = torch.sum((embedding_broad - ze_broad)**2, 2)
        nearest_nei = torch.argmin(distance, 1)
        zq = self.vq_embedding(nearest_nei).permute(0, 3, 1, 2)
        decoder_input = ze + (zq - ze).detach()
        # decode
        x_hat = self.decoder(decoder_input)
        return x_hat, ze, zq


In [3]:
from time import time

n_epochs = 2000
learning_rate = 1e-2
alpha = 1
beta = 0.25

pi = torch.tensor(torch.pi)

def loss_fn(x, x_hat, ze, zq):
    l_reconstruct = F.mse_loss(x, x_hat)
    l_1 = F.mse_loss(ze.detach(), zq)
    l_2 = F.mse_loss(ze, zq.detach())
    loss = l_reconstruct + alpha * l_1 + beta * l_2
    return loss/len(x)

def loss_reconstruction(x, x_hat):
    return F.mse_loss(x, x_hat)/len(x)

def loss_emb(ze, zq):
    l_1 = F.mse_loss(ze.detach(), zq)
    l_2 = F.mse_loss(ze, zq.detach())
    return (alpha * l_1 + beta * l_2)/len(ze)

def train(device, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    begin_time = time()
    # train
    with open('/root/new/VQVAE/loss.txt', 'w') as file:
        for i in range(n_epochs):
            for batch_idx, (x, _) in enumerate(train_loader):
                x = x.to(device)
                x_hat, ze, zq = model(x)
                loss = loss_fn(x, x_hat, ze, zq)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            # estimate loss
            model.eval()
            with torch.no_grad():
                each_epoch = 10
                n_samples = 100
                #train
                indices = torch.randperm(len(train_dataset))[:n_samples]
                x = torch.stack([train_dataset[i][0] for i in indices]).to(device)
                x_hat, ze, zq = model(x)
                loss = loss_fn(x, x_hat, ze, zq)
                loss_recons = loss_reconstruction(x, x_hat)
                loss_embed = loss_emb(ze, zq)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.4f}'.format(i, loss))
                    print('====> Epoch: {} reconst loss: {:.4f}'.format(i, loss_recons))
                    print('====> Epoch: {} embedding loss: {:.4f}'.format(i, loss_embed))
                file.write(str(loss.item()) + ' ' + str(loss_recons.item()) + ' ' + str(loss_embed.item()) + ' ')
                #test
                indices = torch.randperm(len(test_dataset))[:n_samples]
                x = torch.stack([test_dataset[i][0] for i in indices]).to(device)
                x_hat, ze, zq = model(x)
                loss = loss_fn(x, x_hat, ze, zq)
                loss_recons = loss_reconstruction(x, x_hat)
                loss_embed = loss_emb(ze, zq)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.4f}'.format(i, loss))
                    print('====> Epoch: {} reconst loss: {:.4f}'.format(i, loss_recons))
                    print('====> Epoch: {} embedding loss: {:.4f}'.format(i, loss_embed))
                file.write(str(loss.item()) + ' ' + str(loss_recons.item()) + ' ' + str(loss_embed.item()) + '\n')
            
            #time
            if(i % each_epoch == 0):
                training_time = time() - begin_time
                minute = int(training_time // 60)
                second = int(training_time % 60)
                print(f'time loss {minute}:{second}')
        
        torch.save(model, '/root/new/VQVAE/model.pth')
        
    tot_training_time = time() - begin_time
    minute = int(tot_training_time // 60)
    second = int(tot_training_time % 60)
    print(f'total time loss {minute}:{second}')

def initialize_parameters(model):
    for param in model.parameters():
        param.data.normal_(mean=0, std=0.01)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
        
import numpy as np
import pandas as pd

In [ ]:
def main():
    device = 'cuda:0'

    model = VQVAE().to(device)

    # Load the model
    #model.load_state_dict(torch.load('/root/new/VQVAE/model.pth', 'cuda:0'))

    #Choose which to play
    train(device, model)
    #generate(device, model)
    #reconstruct(device, model)
    #latent_space(device, model)
    #print(sum(p.numel() for p in model.parameters() if p.requires_grad))

if __name__ == '__main__':
    main()

In [5]:
device = 'cpu'
model = torch.load('/root/new/VQVAE/model.pth', map_location=device)

In [6]:
# Reconstruction
with torch.no_grad(): 
    
    for batch_idx, (batch_x, _) in enumerate(test_loader):
        true_imgs = batch_x.view(-1, 1, 28, 28)
        save_image(true_imgs, '/root/new/VQVAE/pictures/oring.png', nrow=10)
        break
    
    x = true_imgs.to(device)
    
    reconst_x = model(x)[0]
    reconst_imgs = reconst_x.view(-1, 1, 28, 28)
    save_image(reconst_imgs, '/root/new/VQVAE/pictures/recons.png', nrow=10)

In [ ]:
#drawing

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
        
import numpy as np
import pandas as pd

def moving_average(data, window_size):
    return pd.Series(data).rolling(window=window_size, min_periods=1).mean().values

def drawing():
    plt.clf()
    fig = plt.figure(figsize=(8, 8))
    trains = []
    trains_recon = []
    trains_kl = []
    tests = []
    tests_recon = []
    tests_kl = []
    with open('/root/new/VQVAE/loss.txt', 'r') as file:
        for line in file:
            parts = line.split()
            trains.append(float(parts[0]))
            trains_recon.append(float(parts[1]))
            trains_kl.append(float(parts[2]))
            tests.append(float(parts[3]))
            tests_recon.append(float(parts[4]))
            tests_kl.append(float(parts[5]))
            
    epochs = [len(train_dataset) * i for i in range(1, len(trains) + 1)]
    
    size = 100
    print("Reconstruction Loss: ", sum(trains_recon[-size:])/size)
    print("KL Loss: ", sum(trains_kl[-size:])/size)
    print("Reconstruction Loss: ", sum(tests_recon[-size:])/size)
    print("KL Loss: ", sum(tests_kl[-size:])/size)
    
    window_size = 50
    trains = moving_average(trains, window_size)
    tests = moving_average(tests, window_size)
    trains_recon = moving_average(trains_recon, window_size)
    tests_recon = moving_average(tests_recon, window_size)
    trains_kl = moving_average(trains_kl, window_size)
    tests_kl = moving_average(tests_kl, window_size)
    
    plt.plot(epochs, trains, label='Train Loss')
    plt.plot(epochs, tests, label='Test Loss')
    plt.plot(epochs, trains_recon, label='Train reconstruction Loss')
    plt.plot(epochs, tests_recon, label='Test reconstruction Loss')
    plt.plot(epochs, trains_kl, label='Train embedding Loss')
    plt.plot(epochs, tests_kl, label='Test embedding Loss')
    
    plt.xlabel('Samples')
    plt.ylabel('Loss')
    plt.title('MNIST')
    plt.legend()
    #plt.ylim(-150, -100)
    #plt.xlim(1e5, 1e8)
    plt.xscale('log')
    plt.savefig('/root/new/VQVAE/pictures/train.png',bbox_inches='tight')
    plt.show()

drawing()